In [19]:
import shutil
import os
from sklearn import model_selection
import random
from tqdm import tqdm

In [31]:
# the folder where the images collected using `get_images.py` have been saved
DATAPATH = "images/data_2022-02-13_Oslo_Center"

## Training data

Here we take a subset of images to be labelled and later form the training and testing sets. 

In [30]:
image_files = [f for f in os.listdir(DATAPATH) if f.endswith('.jpg')]

nb_images = 250 # the number of images to copy to another folder for labelling

image_files_to_label = random.sample(image_files, nb_images) 

output_path = DATAPATH + '_labelled'

# make the output directory if is doesn't already exist
isExist = os.path.exists(output_path)
if not isExist:
    # Create a new directory because it does not exist 
    os.makedirs(output_path)
    print(f"{output_path} is created!")

    # copy image subset to a folder for labelling
    for f in tqdm(image_files_to_label):
        shutil.copyfile(DATAPATH + '/' + f, output_path + '/' + f)

images/data_2022-02-13_Oslo_Center_labelled is created!


100%|██████████| 250/250 [00:00<00:00, 336.91it/s]


## Test-train split

Once the new subset of images has been labelled (I used https://github.com/tzutalin/labelImg to label the data according to the YOLO formatting standard) they need to be split up into training and testing sets.

In [38]:
def copy_files(files, output_folder_path, type="train"):
    """takes all file paths of the text files containing labels and places 
    them in the desired folder based on whether they are part of the test or train set

    Args:
        files (list of str): list of filepaths to images labels
        type (str, optional): string. Defaults to "train".
    """
    for f in files:
        # copy over the text files containing the labels
        try:
            shutil.copyfile(output_folder_path + '/' + f, f'./data/labels/{type}/{f}')
        except:
            print(f"Could not copy {f}")

        # copy over the images 
        image_f = f[:-4] + '.jpg'
        try:
            shutil.copyfile(output_folder_path + '/' +  image_f, f'./data/images/{type}/{image_f}')
        except:
            print(f"Could not copy {f}")

In [41]:
text_files = [f for f in os.listdir(output_path) if f.endswith('.txt')]

train, test = model_selection.train_test_split(text_files, test_size=0.2, random_state=42, shuffle=True)

print(f"{len(train)} images for training.")
print(f"{len(test)} images for testing.")

copy_files(train, output_path, type='train')
copy_files(test, output_path, type='validation')

44 images for training.
12 images for testing.
Could not copy classes.txt
